# Upsample to average
Notebook with example for the article on medium.com

In [1]:
import pandas as pd
import io

# Downsample

In [2]:
data="""
CAT|DATE|VALUE
abc|0101|10
abc|0103|20
abc|0107|15"""
df = pd.read_csv(io.StringIO(data), skiprows=1, sep="|", dtype={"DATE":"str"})
df["DATE"] = pd.to_datetime(df["DATE"], format="%m%d")

In [3]:
df.groupby("CAT").resample("2W", on="DATE").agg({"VALUE":["sum","count","mean","min","max","median","first","last"]})

VALUE                                     
                 sum count mean min max median first last
CAT DATE                                                 
abc 1900-01-07    45     3   15  10  20     15    10   15

# Upsample

In [4]:
df.set_index("DATE").groupby("CAT").resample("D").agg({"VALUE":["sum","count","mean","min","max","backfill","bfill","pad","ffill","nearest","first"]})

VALUE                                                           \
                 sum count  mean   min   max backfill bfill pad ffill nearest   
CAT DATE                                                                        
abc 1900-01-01    10     1  10.0  10.0  10.0       10    10  10    10      10   
    1900-01-02     0     0   NaN   NaN   NaN       20    20  10    10      20   
    1900-01-03    20     1  20.0  20.0  20.0       20    20  20    20      20   
    1900-01-04     0     0   NaN   NaN   NaN       15    15  20    20      20   
    1900-01-05     0     0   NaN   NaN   NaN       15    15  20    20      15   
    1900-01-06     0     0   NaN   NaN   NaN       15    15  20    20      15   
    1900-01-07    15     1  15.0  15.0  15.0       15    15  15    15      15   

                      
               first  
CAT DATE              
abc 1900-01-01  10.0  
    1900-01-02   NaN  
    1900-01-03  20.0  
    1900-01-04   NaN  
    1900-01-05   NaN  
    1900-01-06   NaN  
    1900-01-07  15.0

Averaga `mean` didn't really calculate the mean, only filled in the gaps with the `NaN`. To find out the average, you have to use `.group` attribute of the `resample` method and divide the measured value by it. It's more tricky with multiple categories, so let's start with one.

In [5]:
data="""
CAT|DATE|VALUE
abc|0101|10
abc|0103|20
abc|0106|15"""
df = pd.read_csv(io.StringIO(data), skiprows=1, sep="|", dtype={"DATE":"str"})
df["DATE"] = pd.to_datetime(df["DATE"], format="%m%d")

In [6]:
expected="""
CAT|DATE|VALUE
abc|0102|10
abc|0103|10
abc|0104|5
abc|0105|5
abc|0106|5"""

The resample method has two attributes:
    
* `indices` - showing where the input data appeared
* `groups` - indexing each datatime value with its group index.

https://pandas.pydata.org/pandas-docs/stable/reference/resampling.html

In [7]:
# the data came on first, third and sixth of Jan
df.set_index("DATE")\
.resample("D").indices

defaultdict(list,
            {Timestamp('1900-01-01 00:00:00', freq='D'): [0],
             Timestamp('1900-01-03 00:00:00', freq='D'): [1],
             Timestamp('1900-01-06 00:00:00', freq='D'): [2]})

In [8]:
# the groups are 
""" 
    Jan-1 Group 1
    Jan-2 Group 1
    Jan-3 Group 2
    Jan-4 Group 2
    Jan-5 Group 2
    Jan-6 Group 3
"""
df.set_index("DATE")\
.resample("D", label="right").groups

{Timestamp('1900-01-02 00:00:00', freq='D'): 1,
 Timestamp('1900-01-03 00:00:00', freq='D'): 1,
 Timestamp('1900-01-04 00:00:00', freq='D'): 2,
 Timestamp('1900-01-05 00:00:00', freq='D'): 2,
 Timestamp('1900-01-06 00:00:00', freq='D'): 2,
 Timestamp('1900-01-07 00:00:00', freq='D'): 3}

In reality our data from Jan-1 cover unknown period of time, the value from Jan-3 covers the third and second and the last value covers fourth, fifth and sixth of Jan. We can turn the dictionary returned by the `.groups` parameter into a dataframe and shift the group ids by 1 (row) so that data match our scenario.

In [9]:
# Calculate the number of hours in each group
resampled_groups = df.set_index("DATE").resample("D", label="right").groups
# alternatively shift the left labels by 1
#df_groups = pd.DataFrame(df.set_index("DATE").resample("D").groups, index=["group"]).T.shift(1)
df_groups = pd.DataFrame(resampled_groups, index=["group"]).T
df_groups

,group
1900-01-02,1
1900-01-03,1
1900-01-04,2
1900-01-05,2
1900-01-06,2
1900-01-07,3


Knowing which dates belong to each group, let us calculate the number of occurences (days) between each measurement. 

In [10]:
s = df_groups.groupby("group").size()
s.name = "count"
s

group
1    2
2    3
3    1
Name: count, dtype: int64

By joining the counts back to the groups we will see how many days were covered each date so that we can calculate the average - measured value at the end of the period/period lenght.  

In [11]:
s_counts = df_groups.join(s, on="group")["count"]
s_counts

1900-01-02    2
1900-01-03    2
1900-01-04    3
1900-01-05    3
1900-01-06    3
1900-01-07    1
Name: count, dtype: int64

Mering this back to the original dataframe, with backfilled sum allow us to calculate the average. 

In [12]:
res = df.set_index("DATE").resample("D").bfill()
res = res.join(s_counts)
res["average"] = res["VALUE"]/res["count"]
res

,CAT,VALUE,count,average
DATE,,,,
1900-01-01,abc,10,NaN,NaN
1900-01-02,abc,20,2.0,10.0
1900-01-03,abc,20,2.0,10.0
1900-01-04,abc,15,3.0,5.0
1900-01-05,abc,15,3.0,5.0
1900-01-06,abc,15,3.0,5.0


Voila, we have the values split by average to each day. You might say "hurray", but there's one more pitfall. When you have multiple categories, the `.groups` attribute group all of the together, and doesn't create separate groups per category.

In [13]:
data="""
CAT|DATE|VALUE
abc|0101|10
abc|0103|20
abc|0106|15
efg|0101|10
efg|0105|40
efg|0106|12"""

In [14]:
df = pd.read_csv(io.StringIO(data), skiprows=1, sep="|", dtype={"DATE":"str"})
df["DATE"] = pd.to_datetime(df["DATE"], format="%m%d")
df.set_index("DATE").groupby("CAT").resample("D").indices

defaultdict(list,
            {Timestamp('1900-01-01 00:00:00', freq='D'): [0, 1],
             Timestamp('1900-01-03 00:00:00', freq='D'): [2],
             Timestamp('1900-01-05 00:00:00', freq='D'): [3],
             Timestamp('1900-01-06 00:00:00', freq='D'): [4, 5]})

In [15]:
df.set_index("DATE").groupby("CAT").resample("D").groups

{Timestamp('1900-01-01 00:00:00', freq='D'): 2,
 Timestamp('1900-01-02 00:00:00', freq='D'): 2,
 Timestamp('1900-01-03 00:00:00', freq='D'): 3,
 Timestamp('1900-01-04 00:00:00', freq='D'): 3,
 Timestamp('1900-01-05 00:00:00', freq='D'): 4,
 Timestamp('1900-01-06 00:00:00', freq='D'): 6}

For this reason you have to to the average calculation separatelly for each group.

# Putting it all together

In [16]:
data="""
CAT|DATE|VALUE
abc|0101|10
abc|0103|20
abc|0106|15
efg|0101|10
efg|0105|40
efg|0106|12"""
df = pd.read_csv(io.StringIO(data), skiprows=1, sep="|", dtype={"DATE":"str"})
df["DATE"] = pd.to_datetime(df["DATE"], format="%m%d")

updated_df = []
for gr in df["CAT"].unique():
    
    subdf = df[df["CAT"]==gr].set_index("DATE").resample("1D", label="right")
    
    # back fill the data
    res = subdf.bfill()
    
    # Calculate the number of hours in each group
    resampled_groups = subdf.groups
    df_groups = pd.DataFrame(resampled_groups, index=["group"]).T
    df_groups
    
    s = df_groups.groupby("group").size()
    s.name = "count"
    s_counts = df_groups.join(s, on="group")["count"]
    
    res = res.join(s_counts)
    res["daily_average"] = res["VALUE"]/res["count"]
    
    # add to list
    updated_df.append(res)
    
pd.concat(updated_df).dropna()

,CAT,VALUE,count,daily_average
DATE,,,,
1900-01-02,abc,20,2.0,10.0
1900-01-03,abc,20,2.0,10.0
1900-01-04,abc,15,3.0,5.0
1900-01-05,abc,15,3.0,5.0
1900-01-06,abc,15,3.0,5.0
1900-01-02,efg,40,4.0,10.0
1900-01-03,efg,40,4.0,10.0
1900-01-04,efg,40,4.0,10.0
1900-01-05,efg,40,4.0,10.0


In the end, I've dropped all the empty values, because for the first day (the first measurement) we don't know for how long period it's measured.

You have smoothened your data to daily average, but maybe you don't want to feed the model with daily data. You just needed to cover the different times when the inputs are comming, but now want to input only weekly or quarterly data. That is practically useful in case of stock data, when most of the companies report in September while few does in October or November and some really unfriendly ones does it only every sixth months. In that case you can upsample to average monthly values and then caluclate the rolling average for each quarter and feed your model with lower amount of data. 

Let's explore on this short sample of company data

In [17]:
data="""
COM|DATE    |VALUE
abc|20200131|100
abc|20200430|80
abc|20200731|125
abc|20201031|110
efg|20200331|25
efg|20200930|75
ijk|20200331|15
ijk|20200630|45
ijk|20200930|50"""

# read the data into dataframe
df = pd.read_csv(io.StringIO(data), skiprows=1, sep="|", dtype={"DATE    ":"str"}).rename(columns={"DATE    ":"DATE"})
df["DATE"] = pd.to_datetime(df["DATE"], format="%Y%m%d")

In [18]:
updated_df = []
for gr in df["COM"].unique():
    
    subdf = df[df["COM"]==gr].set_index("DATE").resample("M",  label="right", closed="left")
    
    # back fill the data
    res = subdf.bfill()
    
    # Calculate the number of hours in each group
    resampled_groups = subdf.groups
    df_groups = pd.DataFrame(resampled_groups, index=["group"]).T
    df_groups
    
    s = df_groups.groupby("group").size()
    s.name = "count"
    s_counts = df_groups.join(s, on="group")["count"]
    
    res = res.join(s_counts)
    res["monthly_average"] = res["VALUE"]/res["count"]
    
    # add to list
    updated_df.append(res)
    
#av_df = 
av_df = pd.concat(updated_df).dropna()#.reset_index().set_index("DATE")
av_df["3months"] = av_df.groupby("COM")["monthly_average"].rolling(3).sum().values
av_df

,COM,VALUE,count,monthly_average,3months
DATE,,,,,
2020-02-29,abc,80,3.0,26.666667,NaN
2020-03-31,abc,80,3.0,26.666667,NaN
2020-04-30,abc,80,3.0,26.666667,80.000000
2020-05-31,abc,125,3.0,41.666667,95.000000
2020-06-30,abc,125,3.0,41.666667,110.000000
2020-07-31,abc,125,3.0,41.666667,125.000000
2020-08-31,abc,110,3.0,36.666667,120.000000
2020-09-30,abc,110,3.0,36.666667,115.000000
2020-10-31,abc,110,3.0,36.666667,110.000000


In [19]:
# for your model you might want only the quartely data
av_df.loc[[pd.to_datetime("2020-06-30"), pd.to_datetime("2020-09-30")]]

,COM,VALUE,count,monthly_average,3months
DATE,,,,,
2020-06-30,abc,125,3.0,41.666667,110.0
2020-06-30,efg,75,6.0,12.500000,37.5
2020-06-30,ijk,45,3.0,15.000000,45.0
2020-09-30,abc,110,3.0,36.666667,115.0
2020-09-30,efg,75,6.0,12.500000,37.5
2020-09-30,ijk,50,3.0,16.666667,50.0


In [20]:
# for your model you might want only the quartely data
av_df\
.loc[[pd.to_datetime("2020-06-30"), pd.to_datetime("2020-09-30")],["COM","3months"]]\
.pivot(columns="COM" ,values="3months")

COM,abc,efg,ijk
DATE,,,
2020-06-30,110.0,37.5,45.0
2020-09-30,115.0,37.5,50.0


## Why do you need to use label="right", closed="left" with "M"
Daily and monthly resampling works a bit differently with the groups

In [21]:
data="""
COM|DATE    |VALUE
efg|20200331|25
efg|20200930|75
efg|20201031|15"""

# read the data into dataframe
df = pd.read_csv(io.StringIO(data), skiprows=1, sep="|", dtype={"DATE    ":"str"}).rename(columns={"DATE    ":"DATE"})
df["DATE"] = pd.to_datetime(df["DATE"], format="%Y%m%d")

In [22]:
groups = []
for label in ["left","right"]:
    for closed in ["left","right"]:
        resampled_groups = df.set_index("DATE").resample("M", label=label, closed=closed).groups
        groups.append(pd.DataFrame(resampled_groups, index=["l:"+label+"_"+closed]).T)
for label in ["left","right"]:
    resampled_groups = df.set_index("DATE").resample("M", label=label).groups
    groups.append(pd.DataFrame(resampled_groups, index=["l:"+label],dtype="int").T)
out = pd.concat(groups, axis=1).fillna("0").astype("int")

def highlight_cols(s):
    return 'background-color: lightgreen'

out.style.applymap(highlight_cols, subset=pd.IndexSlice[:,["l:right_right","l:right"]])

,l:left_left,l:left_right,l:right_left,l:right_right,l:left,l:right
2020-02-29 00:00:00,0,1,0,0,1,0
2020-03-31 00:00:00,1,1,0,1,1,1
2020-04-30 00:00:00,1,1,1,1,1,1
2020-05-31 00:00:00,1,1,1,1,1,1
2020-06-30 00:00:00,1,1,1,1,1,1
2020-07-31 00:00:00,1,1,1,1,1,1
2020-08-31 00:00:00,1,2,1,1,2,1
2020-09-30 00:00:00,2,3,1,2,3,2
2020-10-31 00:00:00,3,0,2,3,0,3
2020-11-30 00:00:00,0,0,3,0,0,0


In [23]:
data="""
CAT|DATE|VALUE
abc|20200101|10
abc|20200103|20
abc|20200106|15"""
df = pd.read_csv(io.StringIO(data), skiprows=1, sep="|", dtype={"DATE":"str"})
df["DATE"] = pd.to_datetime(df["DATE"], format="%Y%m%d")

In [24]:
groups = []
for label in ["left","right"]:
    for closed in ["left","right"]:
        resampled_groups = df.set_index("DATE").resample("D", label=label, closed=closed).groups
        groups.append(pd.DataFrame(resampled_groups, index=["l:"+label+"_c:"+closed]).T)

for label in ["left","right"]:
    resampled_groups = df.set_index("DATE").resample("D", label=label).groups
    groups.append(pd.DataFrame(resampled_groups, index=["l:"+label]).T)
        
out = pd.concat(groups, axis=1).fillna("0").astype("int")

def highlight_cols(s):
    return 'background-color: lightgreen'

out.style.applymap(highlight_cols, subset=["l:right_c:left","l:right"])

,l:left_c:left,l:left_c:right,l:right_c:left,l:right_c:right,l:left,l:right
2019-12-31 00:00:00,0,1,0,0,0,0
2020-01-01 00:00:00,1,1,0,1,1,0
2020-01-02 00:00:00,1,2,1,1,1,1
2020-01-03 00:00:00,2,2,1,2,2,1
2020-01-04 00:00:00,2,2,2,2,2,2
2020-01-05 00:00:00,2,3,2,2,2,2
2020-01-06 00:00:00,3,0,2,3,3,2
2020-01-07 00:00:00,0,0,3,0,0,3
